In [83]:
import pandas as pd
import numpy as np
import json # library to handle JSON files

!pip install geopy

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

print('Libraries imported.')

Libraries imported.


# First Part
This part contains extracting the data from Wikipedia and processing it into a dataframe by removing 'Not Assigned' Boroughs and formating the Neighborhood column according to specifications.

The only assumption made is that there are no Neighboorhoods that are not named (except for those Not Assigned Boroughs) and that can be seen in the Wikipedia page. 

In [103]:
url_html='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df = pd.read_html(url_html) #load into a data frame
df = df[0]
df['Borough'].value_counts() #Find how many boroughs are not assigned

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
York                 5
East York            5
Mississauga          1
Name: Borough, dtype: int64

In [107]:
df.dropna(inplace = True) #drop 'Not Assigned' boroughs

In [108]:
#Format DF
df.reset_index(inplace=True)
df.drop(columns = 'index',inplace =True)

In [109]:
# Create new columns and set them as the dataframe's columns
columns = ['PostalCode', 'Borough', 'Neighborhood']
df.columns = columns

In [110]:
df.dtypes

PostalCode      object
Borough         object
Neighborhood    object
dtype: object

In [112]:
hood = df['Neighborhood']
neighborhood = hood.str.replace('/',',') # Replace parentheses with commas as requested.
neighborhood = pd.DataFrame(neighborhood)

In [113]:
df.drop(columns = 'Neighborhood',inplace =True) #drop original Neighborhood column that contains parentheses to separate neighborhoods

df1 =pd.concat([df,neighborhood], axis = 1, sort=False) #Join orignal dataframe to the neighborhood dataframe, which contains the changes made

Below is the .shape method to print the number of rows of your dataframe.

In [115]:
print("Shape of DataFrame: {}".format(df1.shape)) 
df1.head(10)

Shape of DataFrame: (103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


# Second Part
In this part, the latitudes and longitudes of the different locations are retrieved, by the postal code, using Geocoder. 

In [93]:
!pip install geocoder==1.5. #install geocoder

In [94]:
col_names = ['Latitude', 'Longitude']
cols = pd.DataFrame(columns=col_names)

In [95]:
import geocoder # import geocoder


for code in df['PostalCode']: #For loop to go through all the postal code values
    
    zip_code = code #initalizating variable for postal code
    
    lat_lng_coords = None 
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(zip_code))
        lat_lng_coords = g.latlng

    
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    
    cols = cols.append({'Latitude': latitude,
                       'Longitude': longitude}, ignore_index=True) #append all the values into the dataframe

In [96]:
cols.head(20)

,Latitude,Longitude
0,43.752935,-79.335641
1,43.728102,-79.311890
2,43.650964,-79.353041
3,43.723265,-79.451211
4,43.661790,-79.389390
5,43.667481,-79.528953
6,43.808626,-79.189913
7,43.748900,-79.357220
8,43.707193,-79.311529
9,43.657491,-79.377529


In [97]:
df2 = pd.concat([df1,cols],axis = 1, sort=False) #concatenate the coordinates dataframe and the locations dataframe

In [98]:
df2.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.661790,-79.389390
5,M9A,Etobicoke,Islington Avenue,43.667481,-79.528953
6,M1B,Scarborough,"Malvern , Rouge",43.808626,-79.189913
7,M3B,North York,Don Mills,43.748900,-79.357220
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.707193,-79.311529
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529
